In [ ]:
pip install beautifulsoup4

In [ ]:
import requests
from bs4 import BeautifulSoup

# URL of the law page
url = "https://www.legifrance.gouv.fr/codes/section_lc/LEGITEXT000005634379/LEGISCTA000006113738/#LEGISCTA000006113738"
# Send a GET request to fetch the webpage
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Parse the HTML content of the page
    soup = BeautifulSoup(response.content, 'html.parser')

    # Extract document name
    document_name_tag = soup.find('h2', class_='Title-of-Act')
    if document_name_tag:
        document_name = document_name_tag.get_text(strip=True)
        print(f"Document Name: {document_name}")
    else:
        print("Document name not found")

    # Now let's print out a portion of the HTML to inspect the structure around the sections
    print("\nRaw HTML Sample:\n")
    print(soup.prettify()[25000:70000])

else:
    print(f"Failed to retrieve the page. Status code: {response.status_code}")

Document name not found

Raw HTML Sample:

   </span>
              </a>
             </li>
            </ul>
           </li>
           <li class="menu-item expanded small-menu">
            <a class="dropdown-toggle" data-drupal-link-system-path="taxonomy/term/6" data-hover="dropdown" href="/contenu/menu/droit-et-jurisprudence-de-l-union-europeenne" role="button">
             <span>
              Droit et jurisprudence
              <br/>
              de l'Union européenne
             </span>
            </a>
            <ul class="sub-menu level-two">
             <li class="menu-item first">
              <a href="https://eur-lex.europa.eu/oj/direct-access.html?locale=fr" target="_blank" title="Journal officiel de l'Union européenne">
               <span>
                Journal officiel de l'Union européenne
               </span>
              </a>
             </li>
             <li class="menu-item small-menu">
              <a href="https://europa.eu/european-union/law/fi

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

# List of URLs to scrape
urls = pd.read_csv('/content/links_national_france.csv', header=None)
urls = urls[0].tolist()
#urls = ["https://www.legifrance.gouv.fr/codes/section_lc/LEGITEXT000005634379/LEGISCTA000006113738/#LEGISCTA000006113738"]

# Initialize a DataFrame to hold the concatenated data
all_data = pd.DataFrame()

# Function to scrape a single URL
def scrape_url(url):
    try:
        response = requests.get(url)
        if response.status_code == 200:
            print(f"Successfully accessed: {url}")
            # Parse the HTML content of the page
            soup = BeautifulSoup(response.content, 'html.parser')

            # Extract document name from <div class="summary-header print-invisible">
            document_name_tag = soup.find('div', class_='summary-header print-invisible')
            if document_name_tag:
                document_name = document_name_tag.get_text(strip=True)
            else:
                document_name = "Unknown"
                print(f"Warning: Document name not found on {url}")

            # Extract document date from <h2 class="vigor-title">
            document_date_tag = soup.find('h2', class_='vigor-title')
            if document_date_tag:
                document_date = document_date_tag.get_text(strip=True)
            else:
                document_date = "Unknown"
                print(f"Warning: Document date not found on {url}")

            # Initialize a list to hold the rows of this URL
            data = []

            # Find all article names (titles) with <p class="name-article">
            articles = soup.find_all('p', class_='name-article')
            if not articles:
                print(f"Warning: No articles found on {url}")

            for article in articles:
                try:
                    # Extract article name (e.g., "Article L123-50")
                    article_name_tag = article.find('a')
                    if article_name_tag:
                        article_name = article_name_tag.get_text(strip=True)
                    else:
                        article_name = "Unknown"
                        print(f"Warning: Article name not found on {url}")

                    # Avoid elements with the "abrogated" class
                    if "abrogated" in article.get('class', []):
                        print(f"Skipping abrogated article: {article_name}")
                        continue

                    # Extract the article's content from the following <div class="content">
                    content_div = article.find_next('div', class_='content')
                    if content_div:
                        article_text = ' '.join(content_div.stripped_strings)
                    else:
                        article_text = "No content available"
                        print(f"Warning: No content found for article {article_name}")

                    # Save the extracted data
                    row = {
                        'document_name': document_name,
                        'document_date': document_date,
                        'document_section_title': article_name,
                        'text': article_text
                    }
                    data.append(row)

                except Exception as article_error:
                    print(f"Error processing article on {url}: {article_error}")

            # Convert the scraped data into a DataFrame
            if data:
                return pd.DataFrame(data)
            else:
                print(f"No data extracted from {url}")
                return pd.DataFrame()

    except Exception as e:
        print(f"Error scraping {url}: {e}")
        return pd.DataFrame()

# Loop through each URL with a delay between requests
for url in urls:
    print(f"Scraping: {url}")
    df = scrape_url(url)

    if not df.empty:
        # Concatenate the new data with the existing DataFrame
        all_data = pd.concat([all_data, df], ignore_index=True)

    # Pause between requests to avoid overwhelming the server
    time.sleep(5)  # Adjust the pause time if needed

# Save the final concatenated DataFrame to an Excel file with UTF-8 encoding
all_data.to_excel('france_national_laws.xlsx', index=False)

# Show the result
print(all_data)



Se han truncado las últimas 5000 líneas del flujo de salida.
Skipping abrogated article: Unknown
Skipping abrogated article: Unknown
Skipping abrogated article: Unknown
Skipping abrogated article: Unknown
Skipping abrogated article: Unknown
Skipping abrogated article: Unknown
Skipping abrogated article: Unknown
Skipping abrogated article: Unknown
Skipping abrogated article: Unknown
Skipping abrogated article: Unknown
Skipping abrogated article: Unknown
Skipping abrogated article: Unknown
Skipping abrogated article: Unknown
Skipping abrogated article: Unknown
Skipping abrogated article: Unknown
Skipping abrogated article: Unknown
Skipping abrogated article: Unknown
Skipping abrogated article: Unknown
Skipping abrogated article: Unknown
Skipping abrogated article: Unknown
Skipping abrogated article: Unknown
Skipping abrogated article: Unknown
Skipping abrogated article: Unknown
Skipping abrogated article: Unknown
Skipping abrogated article: Unknown
Skipping abrogated article: Unknown
Ski